# Cross validation done wrong

Cross validation is an essential tool in statistical learning to estimate the accuracy of your machine learning algorithm. Despite its great power it also exposes some fundamental risk when done wrong which may terribly bias your accuracy estimate.

In this blog post I'll demonstrate using the Python scikit-learn framework how to avoid the biggest and most common pitfall of cross validation in your experiments.

## Theory first

Cross validation involves randomly dividing the set of observations into `k` groups (or folds) of approximately equal size. The first fold is treated as a validation set, and the machine learning algorithm is trained on the remaining `k-1` folds. The mean squared error is then computed on the held-out fold. This procedure is repeated k times; each time, a different group of observations is treated as a validation set.

This process results in k estimates of the MSE quantity, namely $MSE_1$, $MSE_2$,...$MSE_k$. The cross validation estimate for the MSE is then computed by simply averaging these values: 
$$CV_{(k)} = 1/k \sum_{i=1}^k MSE_i$$

This value is an _estimate_, say $\hat{MSE}$, of the real $MSE$ and our goal it to make this estimate as accurate as possible.

## Hands on

Let's now have a look at one of the most typical mistakes when using cross validation. When cross validation is done wrong the result is that the estimate $\hat{MSE}$ does not reflect its real value. In other words, you may think that you just found a perfect machine learning algorithm with incredibly low $MSE$, while in reality you simply wrongly applied CV.

I'll first show you - hands on - a wrong application of cross validation and then we will fix it together. To make things simple let's first generate some random data and let's pretend that we want to build a machine learning algorithm to predict the outcome.

### Dataset generation

In [42]:
# Import pandas
import pandas as pd
from pandas import *

# Import scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import *
from sklearn.metrics import *
import random

I'll first generate a dataset of $100$ entries. Each entry has $10.000$ features. But, why so many? To demonstrate our issue I need to generate some correlation between our inputs and output which is purely casual. You'll understand _the why_ later in this post.

In [2]:
features = np.random.randint(0,10,size=[100,10000])
target = np.random.randint(0,2,size=100)

In [3]:
df = DataFrame(features)
df['target'] = target
df.head()

,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,target
0,1,2,0,1,6,0,2,6,9,3,...,7,6,0,1,4,1,0,8,3,1
1,4,7,9,2,4,9,9,9,4,7,...,9,1,0,0,2,6,2,1,6,0
2,1,1,1,1,1,6,4,0,9,5,...,8,4,5,7,6,5,4,8,5,1
3,2,0,5,2,5,9,7,9,2,5,...,5,1,5,1,3,3,3,8,3,1
4,6,3,0,0,8,0,9,9,1,4,...,6,4,2,1,5,7,9,7,5,1


### Feature selection

At this point we would like to know what are the features that are more useful to train our predictor. This is called _feature selection_. The simplest approach to do that is to find which of the $10.000$ features in our input is mostly correlated the target. Using `pandas` this is very easy to do thanks to the [`corr()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) function. We run `corr()` on our dataframe, we order the correlation values, and we pick the first two features. 

In [8]:
corr = df.corr()['target'][df.corr()['target'] < 1].abs()
corr.sort(ascending=False)

In [9]:
corr.head()

3122    0.392430
830     0.367405
8374    0.351462
9801    0.341806
5585    0.336950
Name: target, dtype: float64

### Start the training

Great! Out of the $10.000$ features we have been able to select two of them, i.e. feature number $3122$ and $830$ that have a $0.36$ and $0.39$ correlation with the output. At this point let's just drop all the other columns and use these two features to train a simple `LogisticRegression`. We then use scikit-learn `cross_val_score` to produce $\hat{MSE}$.

In [10]:
features = corr.index[[0,1]].values

In [13]:
training_input  = df[features].values
training_output = df['target']

In [14]:
logreg = LogisticRegression()

In [27]:
# scikit learn return the negative value for MSE 
# http://stackoverflow.com/questions/21443865/scikit-learn-cross-validation-negative-values-with-mean-squared-error
mse_estimate = -1 * cross_val_score(logreg, training_input, training_output, cv=10, scoring='mean_squared_error')

In [28]:
mse_estimate

array([ 0.45454545,  0.27272727,  0.27272727,  0.5       ,  0.4       ,
        0.2       ,  0.2       ,  0.44444444,  0.33333333,  0.22222222])

In [29]:
DataFrame(mse_estimate).mean()

0    0.33
dtype: float64

### Knowledge leaking

According to the previous estimate we built a system that can predict a random noise target from a random noise input with a $MSE$ of just $0.33$. The result is, as you can expect, wrong. But why? 

The reason is rather counterintuitive and this is why this mistake is so common. When we applied the feature selection we used information from both the training set and the test sets used for the cross validation, i.e. the correlation values. As a consequence our LogisticRegression knew information in the test sets that were supposed to be hidden to it. In fact, when you are computing $MSE_i$ in the i-th iteration of the cross validation you should be using only the information on the training fold, and nothing should come from the test fold. In our case the model did indeed have information from the test fold, i.e. the top correlated features. I think the term **knowledge leaking** express this concept fairly well. 

The schema that follows shows you how the knowledge leaked into the LogisticRegression because the feature selection has been applied beforehand the cross validation procedure started. The model know something about the data highlighted in yellow that it shoulnd't know, its top correlated features.

<img src="cv_example.png" width="80%" height="80%">
Figure 1. _ The exposed knowledge leaking. The LogisticRegression knows the top correlated features of the entire dataset (hence including test folds) because of the initial correlation operation, whilst it should be exposed only to the training fold information._

### Proof that our model is biased
To check that we were actually wrong let's do the following: 
* Take out a portion of the data set (take_out_set).
* Train the LogisticRegression on the remaining data using the same feature selection we did before.
* After the training is done check the $MSE$ on the take_out_set. 

Is the $MSE$ on the take_out_set similar to the $\hat{MSE}$ we estimated with the CV?  
The answer is no, and we got a much more reasonable $MSE$ of $0.53$ that is much higher than the $\hat{MSE}$ of $0.33$. 

In [30]:
take_out_set = df.ix[random.sample(df.index, 30)]

In [31]:
training_set = df[~(df.isin(take_out_set)).all(axis=1)]

In [32]:
corr = training_set.corr()['target'][df.corr()['target'] < 1].abs()
corr.sort(ascending=False)
features = corr.index[[0,1]].values

In [33]:
training_input  = training_set[features].values
training_output = training_set['target']

In [34]:
logreg = LogisticRegression()

In [38]:
logreg.fit(training_input, training_output)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [39]:
y_take_out = logreg.predict(take_out_set[features])

In [43]:
mean_squared_error(take_out_set.target, y_take_out)

0.53333333333333333

## Cross validation done right

In the previous section we have seen that if you inject test knowledge in your model your cross validation procedure will be biased. To avoid this let's compute the features correlation during each cross validation batch. The difference is that now the features correlation will use only the information in the training fold instead of the entire dataset. That's a key insight causing the bias we saw previously. The following graph shows you the revisited procedure. This time we got a realistic $\hat{MSE}$ of $0.53$ that confirms the data is randomly distributed.

<img src="cv_example_2.png" width="80%" height="80%">
Figure 2. _Revisited cross validation workflow with the correlation step performed for each of the K fold train/test data folds._

In [62]:
kf = KFold(df.shape[0], n_folds=10)
mse = []
fold_count = 0
for train, test in kf:
    print("Processing fold %s" % fold_count)
    train_fold = df.ix[train]
    test_fold = df.ix[test]
    
    # find best features
    corr = train_fold.corr()['target'][train_fold.corr()['target'] < 1].abs()
    corr.sort(ascending=False)
    features = corr.index[[0,1]].values
    
    # Get training examples
    train_fold_input  = train_fold[features].values
    train_fold_output = train_fold['target']
    
    # Fit logistic regression
    logreg = LogisticRegression()
    logreg.fit(train_fold_input, train_fold_output)
    
    # Check MSE on test set
    pred = logreg.predict(test_fold[features])
    mse.append(mean_squared_error(test_fold.target, pred))
    
    # Done with the fold
    fold_count += 1

print(DataFrame(mse).mean())

Processing fold 0
Processing fold 1
Processing fold 2
Processing fold 3
Processing fold 4
Processing fold 5
Processing fold 6
Processing fold 7
Processing fold 8
Processing fold 9
0    0.53
dtype: float64


In [64]:
DataFrame(mse).mean()

0    0.53
dtype: float64

## Conclusion

We have seen how doing features engineering at the wrong step when using cross validation can terribly bias the `MSE` estimate of your machine learning algorithm. We have also seen how to correctly apply cross validation by simply moving one step down the feature engineering such that the knowledge from the test data does not leak in our learning procedure. If you want know more about cross validation and the its tradeoffs both R. Kohavi and Y. Bengio with Y. Grandvalet wrote on this topic.


### References

[1] [Lecture 1 on cross validation - Statistical Learning @ Stanford](https://www.youtube.com/watch?v=nZAM5OXrktY)  
[2] [Lecture 2 on cross validation - Statistical Learning @ Stanford](https://www.youtube.com/watch?v=S06JpVoNaA0)  
[3] [Ron Kohavi. A study of cross-validation and bootstrap for accuracy estimation and model selection](http://dl.acm.org/citation.cfm?id=1643047)  
[4] [Y. Bengio and Y. Grandvalet. No unbiased estimator of the variance of k-fold cross-validation](http://dl.acm.org/citation.cfm?id=1044695)
